<a href="https://colab.research.google.com/github/ryzhokhina/Tensorflow-Dev-Exam-Preparation/blob/main/cat_vs_dogs_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
devices = tf.config.list_physical_devices()
print(devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  786M  100  786M    0     0   168M      0  0:00:04  0:00:04 --:--:--  185M


In [4]:
!unzip -q kagglecatsanddogs_5340.zip
!ls

 CDLA-Permissive-2.0.pdf      PetImages        sample_data
 kagglecatsanddogs_5340.zip  'readme[1].txt'


In [5]:
sourse_path = 'PetImages'

In [6]:
os.listdir(sourse_path)

['Dog', 'Cat']

In [7]:
import os

num_skipped = 0
skiped_files = []
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join(sourse_path, folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            skiped_files.append(fpath)
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 1590 images


In [8]:
cat_dir = os.path.join(sourse_path, 'Cat')
dog_dir = os.path.join(sourse_path, 'Dog')
print(f'Count of image in Cat folder {len(os.listdir(cat_dir))}')
print(f'Count of image in Dog folder {len(os.listdir(dog_dir))}')

Count of image in Cat folder 11742
Count of image in Dog folder 11670


In [9]:
IMG_SIZE = 224

train_raw_ds, val_raw_ds = tf.keras.utils.image_dataset_from_directory(
    sourse_path,
    batch_size = 128,
    labels='inferred',
    shuffle=True,
    seed=1234,
    validation_split=0.2,
    subset='both',
    image_size = (IMG_SIZE, IMG_SIZE)
)

Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Using 4682 files for validation.


In [10]:
# data augmentation

augnment = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2)
])

In [11]:
train_ds = train_raw_ds.map(
    lambda x, y: (augnment(x), y)
).shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

val_ds = val_raw_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

In [12]:
# create model using EfficientNetB0

def create_model() -> tf.keras.Model:
    base_model = tf.keras.applications.EfficientNetB0(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        weights='imagenet',
        include_top=False)
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model


In [13]:
effientnet_model = create_model()


16705208/16705208 [==============================] - 2s 0us/step


In [14]:
effientnet_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [15]:
ES = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = effientnet_model.fit(
    train_ds,
    epochs=4,
    validation_data=val_ds,
    #callbacks=[ES]
)


Epoch 1/4
147/147 [==============================] - 260s 320ms/step - loss: 0.2160 - accuracy: 0.9249 - val_loss: 0.0593 - val_accuracy: 0.9878
Epoch 2/4
147/147 [==============================] - 244s 297ms/step - loss: 0.0920 - accuracy: 0.9696 - val_loss: 0.0409 - val_accuracy: 0.9893
Epoch 3/4
147/147 [==============================] - 244s 298ms/step - loss: 0.0745 - accuracy: 0.9737 - val_loss: 0.0340 - val_accuracy: 0.9908
Epoch 4/4
147/147 [==============================] - 245s 298ms/step - loss: 0.0702 - accuracy: 0.9746 - val_loss: 0.0310 - val_accuracy: 0.9910


In [16]:
effientnet_model.save('effientnet_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
effientnet_model.evaluate(val_ds)




37/37 [==============================] - 9s 230ms/step - loss: 0.0310 - accuracy: 0.9910


[0.030968833714723587, 0.9910295009613037]